In [15]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
import json
import pickle

import src.bikewaysim_functions as bikewaysim_functions

import sys
sys.path.insert(0,str(Path.cwd().parent))
import file_structure_setup
config = file_structure_setup.filepaths()

In [16]:
#study area
studyarea = gpd.read_file(config['studyarea_fp'])

arc_fp = Path("D:\RAW\Atlanta_Regional_Comission")

# how many tazs
all_tazs = gpd.read_file(arc_fp/"Model_Traffic_Analysis_Zones_2020.geojson",ignore_geometry=True).shape[0]

#import taz geo
tazs = gpd.read_file(arc_fp/"Model_Traffic_Analysis_Zones_2020.geojson",mask=studyarea)[['MTAZ10','geometry']]
tazs.rename(columns={'MTAZ10':'tazid'},inplace=True)
print(tazs.shape[0],'out of the',all_tazs,'TAZs')
tazs_original = tazs.copy()
# tazs['centroid_geo'] = tazs.geometry.centroid
# tazs.set_geometry('centroid_geo',inplace=True)

1327 out of the 5922 TAZs


In [17]:
# import network nodes
nodes = gpd.read_file(config['network_fp']/"final_network.gpkg",layer="nodes")

with (config['calibration_fp']/"calibration_network.pkl").open('rb') as fh:
    links,turns = pickle.load(fh)
del turns

nodes0 = links['A'].append(links['B'])
nodes = nodes[nodes['N'].isin(nodes0)]

In [18]:
# snap tazs to network
tazs = bikewaysim_functions.snap_ods_to_network(tazs,nodes)

# only keep if dist is less than or equal to 500 feet
tazs = tazs[tazs['dist']<=500]

In [19]:
#import arc trip data
tripData0 = pd.read_csv(arc_fp/"rtp_amd6_2030_tripData/tripData.csv",usecols=['orig_taz','dest_taz','distance'])
tripData1 = pd.read_csv(arc_fp/"rtp_amd6_2030_tripData/tripData_1.csv",usecols=['orig_taz','dest_taz','distance'])
tripData2 = pd.read_csv(arc_fp/"rtp_amd6_2030_tripData/tripData_2.csv",usecols=['orig_taz','dest_taz','distance'])
tripData = pd.concat([tripData0,tripData1,tripData2])
del tripData0, tripData1, tripData2

In [20]:
#remove trips above 3 miles
tripData = tripData[tripData['distance'] <= 3]
tripData = tripData.groupby(['orig_taz','dest_taz']).size().reset_index()
tripData.columns = ['orig_taz','dest_taz','trips']
tripData = tripData[tripData['orig_taz'].isin(tazs['tazid']) & tripData['dest_taz'].isin(tazs['tazid'])]
#remove intrazonal trilps
tripData = tripData[tripData['orig_taz'] != tripData['dest_taz']]

In [21]:
print(tripData['trips'].sum(),'trips across',tripData.shape[0],'unique OD pairs')#,tripData['hh_id'].nunique(),'persons')

3579964 trips across 126641 unique OD pairs


In [22]:
snapped_ods = dict(zip(tazs['tazid'],tazs['N']))
tripData['orig_N'] = tripData['orig_taz'].map(snapped_ods)
tripData['dest_N'] = tripData['dest_taz'].map(snapped_ods)

In [23]:
trip_origins_per_taz = tripData.groupby('orig_taz')['trips'].sum().reset_index()
trip_origins_per_taz.columns = ['tazid','trip_origins']

In [25]:
# no trips
no_trips = tazs[tazs['tazid'].isin(trip_origins_per_taz['tazid'])==False]
no_trips

,tazid,geometry,N,dist
409,661.0,"MULTIPOLYGON (((2230772.488 1363811.126, 22307...",1581270430,184.028907
689,942.0,"MULTIPOLYGON (((2217757.384 1350921.300, 22176...",10231260073,212.632775
846,1134.0,"MULTIPOLYGON (((2210898.541 1324704.872, 22108...",69154389,335.047867
948,1389.0,"MULTIPOLYGON (((2253129.620 1413387.880, 22530...",5426565661,120.181199


In [26]:
# too few to matter
# no_trips.to_crs(config['projected_crs_epsg']).to_file(config['bikewaysim_fp']/'map_layers.gpkg',layer='no_trip_tazs')

In [10]:
tazs = tazs.merge(trip_origins_per_taz,on='tazid')

In [11]:
# trip origins per TAZ

#export the taz areas
tazs.to_crs(config['projected_crs_epsg']).to_file(config['bikewaysim_fp']/'map_layers.gpkg',layer='taz_polygons')
#export the taz centroids
taz_centroids = tazs.copy().to_crs(config['projected_crs_epsg'])
taz_centroids.geometry = taz_centroids.geometry.centroid
taz_centroids.to_crs(config['projected_crs_epsg']).to_file(config['bikewaysim_fp']/'map_layers.gpkg',layer='taz_centroids')

In [12]:
#export
tripData.to_csv(config['bikewaysim_fp']/'od_matrix.csv',index=False)

In [13]:
tripData0 = pd.read_csv(arc_fp/"rtp_amd6_2030_tripData/tripData.csv",nrows=10)

In [14]:
tripData0.columns

Index(['hh_id', 'person_id', 'person_num', 'tour_id', 'stop_id', 'inbound',
       'tour_category', 'tour_purpose', 'orig_purpose', 'dest_purpose',
       'orig_taz', 'dest_taz', 'orig_walk_segment', 'dest_walk_segment',
       'parking_taz', 'depart_period', 'trip_mode', 'tour_mode',
       'num_participants', 'tour_participants', 'tour_start_period',
       'tour_id_uniq', 'trip_id', 'orig_purpose_start_period', 'home_taz',
       'hh_income', 'hh_autos', 'hh_fp_choice', 'hh_inc_bin', 'hh_size',
       'hh_wkrs', 'hh_auto_suff', 'age', 'gender', 'person_type',
       'activity_pattern', 'tour_mode_name', 'trip_mode_name', 'travel_time',
       'distance', 'cost', 'first_wait_time', 'xfer_wait_time',
       'walk_aux_time', 'auto_aux_time', 'boardings', 'ivt_local_bus',
       'ivt_premium_modes'],
      dtype='object')